<a href="https://colab.research.google.com/github/puneetsinha/Python_notebooks/blob/master/Nuitee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import Levenshtein
import jellyfish

# create a pandas dataframe

df = pd.read_csv('rooms_dataset.csv')
df.head()

,provider_id,provider_room_id,provider_room_name,hotel_id,Name
0,24.0,201338332,"Comfort Room, 1 Queen Bed, Non Smoking, Refrig...",21227.0,Best Western Premier Hotel Aristocrate
1,24.0,201338333,"Comfort Room, 2 Queen Beds, Non Smoking, Refri...",21227.0,Best Western Premier Hotel Aristocrate
2,24.0,201338336,"Design Room, 1 King Bed, Non Smoking, Balcony",21227.0,Best Western Premier Hotel Aristocrate
3,24.0,201338337,"Design Room, 1 King Bed, Non Smoking, Jetted Tub",21227.0,Best Western Premier Hotel Aristocrate
4,24.0,201338334,"Design Room, 1 King Bed, Non Smoking, Refriger...",21227.0,Best Western Premier Hotel Aristocrate


In [41]:
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
provider_id,provider_id,1.679253
provider_room_id,provider_room_id,0.000000
provider_room_name,provider_room_name,0.665739
hotel_id,hotel_id,0.953895
Name,Name,0.953895


In [42]:
print(df.shape)
df.dropna(inplace=True)
print(df.shape)

(10064, 5)
(9860, 5)


In [10]:
df.dtypes

provider_id           float64
provider_room_id        int64
provider_room_name     object
hotel_id              float64
Name                   object
dtype: object

In [26]:
import re

def normalize_text(text):
    # Replace non-alphanumeric characters (excluding digits) with spaces
    normalized_text = re.sub('[^a-zA-Z0-9\s\d]+', ' ', text)
    # Replace multiple spaces with a single space
    normalized_text = re.sub('\s+', ' ', normalized_text)
    # Remove leading/trailing spaces
    normalized_text = normalized_text.lower().strip()
    return normalized_text

In [35]:
import warnings
warnings.filterwarnings('ignore')
hoteldf=df[df.hotel_id==21227]
similarity_scores = {} # Create a dictionary to store the similarity scores for each provider_id
for provider_id1 in hoteldf['provider_id'].unique():
  for provider_id2 in hoteldf['provider_id'].unique():
    # Dont look for same and look for other providers
    if provider_id1 != provider_id2:
        # Calculate the similarity score for each pair of provider_room_ids
        similarity_scores[(provider_id1, provider_id2)] = []
        for text1 in df[df.provider_id==provider_id1]['provider_room_name'].tolist():
            for text2 in df[df.provider_id==provider_id2]['provider_room_name'].tolist():
              # print(provider_id1,provider_id2)
              # print(normalize_text(text1), normalize_text(text2))
              similarity_lev = fuzz.ratio(normalize_text(text1), normalize_text(text2))
              similarity_jw = jellyfish.jaro_winkler(normalize_text(text1), normalize_text(text2))
              similarity_scores[(provider_id1, provider_id2)].append((text1, text2,hoteldf[df.provider_room_name==text1]['provider_room_id'].iloc[0],
                                                                      hoteldf[df.provider_room_name==text2]['provider_room_id'].iloc[0], similarity_lev, similarity_jw))
                



# Convert the similarity scores dictionary to a dataframe
similarity_df = pd.DataFrame(columns=['provider_id1', 'provider_id2', 'provider_room_id 1', 'provider_room_id 2','provider_room_name 1',
                                      'provider_room_name 2', 'similarity (Levenshtein Distance)', 'similarity (Jaro-Winkler Distance)'])
for (provider_id1, provider_id2), scores in similarity_scores.items():
    for score in scores:
        similarity_df = similarity_df.append({'provider_id1': provider_id1,
                                              'provider_id2': provider_id2,
                                              'provider_room_id 1': score[0],
                                              'provider_room_id 2': score[1],
                                              'provider_room_name 1' :score[2],
                                              'provider_room_name 2' :score[3],
                                              'similarity (Levenshtein Distance)': score[4],
                                              'similarity (Jaro-Winkler Distance)': score[5]}, ignore_index=True)

In [39]:
print(hoteldf.shape,similarity_df.shape)

(29, 6) (696, 8)


In [44]:
#'1 King Bed, Non - Smoking, Signature Design Room, 42 Inch Lcd Television, Refrigerator, High Speed Internet Access, Work Desk'

normalize_text('1 King Bed, Non-Smoking, Signature Design Room, 42 Inch Lcd Television, Refrigerator, High Speed Internet Access, Work Desk')

'1 king bed non smoking signature design room 42 inch lcd television refrigerator high speed internet access work desk'

In [36]:
similarity_df

,provider_id1,provider_id2,provider_room_id 1,provider_room_id 2,provider_room_name 1,provider_room_name 2,similarity (Levenshtein Distance),similarity (Jaro-Winkler Distance)
0,24.0,41.0,"Comfort Room, 1 Queen Bed, Non Smoking, Refrig...","1 King Bed, Non-Smoking, Signature Design Room...",201338332,85892736,39,0.612105
1,24.0,41.0,"Comfort Room, 1 Queen Bed, Non Smoking, Refrig...","1 King Bed, Non-Smoking, Signature Design Room...",201338332,74225225,39,0.627418
2,24.0,41.0,"Comfort Room, 1 Queen Bed, Non Smoking, Refrig...","1 Queen Bed, Non-Smoking, Comfort Room, 42 Inc...",201338332,66905107,48,0.666854
3,24.0,41.0,"Comfort Room, 1 Queen Bed, Non Smoking, Refrig...","2 Queen Beds, Non-Smoking, Comfort Room, 42 In...",201338332,17715441,48,0.652541
4,24.0,41.0,"Comfort Room, 1 Queen Bed, Non Smoking, Refrig...","2 Queen Beds, Non-Smoking, Signature Design Ro...",201338332,65293496,42,0.617358
...,...,...,...,...,...,...,...,...
691,282.0,259.0,2 Queen Beds - Nonsmoking Room,"1 King Bed, Non-Smoking, Signature Design Room...",83180314,85892736,30,0.583537
692,282.0,259.0,2 Queen Beds - Nonsmoking Room,"1 King Bed, Non-Smoking, Signature Design Room...",83180314,74225225,31,0.584875
693,282.0,259.0,2 Queen Beds - Nonsmoking Room,"1 Queen Bed, Non-Smoking, Comfort Room, 42 Inc...",83180314,66905107,38,0.597532
694,282.0,259.0,2 Queen Beds - Nonsmoking Room,"2 Queen Beds, Non-Smoking, Comfort Room, 42 In...",83180314,17715441,41,0.691991


In [46]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 27.5 MB/s eta 0:00:00


In [49]:
from transformers import pipeline

hotel_classifier = pipeline('text-classification', model='distilbert-base-uncased-finetuned-sst-2-english')

def classify_hotel(text):
    result = hotel_classifier(text)[0]
    if result['label'] == 'LABEL_1':
        return True
    else:
        return False

if classify_hotel("Welcome to Hilton Hotel!"):
    context = "Hilton Hotel is located in the heart of downtown, with easy access to many restaurants, shops, and attractions."
else:
    context = "Sorry, I couldn't identify the hotel you mentioned."

qa_bot = pipeline('question-answering', model='distilbert-base-uncased-distilled-squad')

while True:
    message = input("You: ")
    if "location" in message.lower():
        result = qa_bot(question="What is the location of the hotel?", context=context)
        response = result['answer']
    elif "amenities" in message.lower():
        result = qa_bot(question="What are the amenities provided by the hotel?", context=context)
        response = result['answer']
    elif "check-in" in message.lower():
        result = qa_bot(question="What is the check-in time of the hotel?", context=context)
        response = result['answer']
    else:
        response = "Sorry, I don't understand your question."
    print("Bot:", response)

You: What is the location of the hotel?
Bot: I couldn't identify the hotel
You: hilton
Bot: Sorry, I don't understand your question.


KeyboardInterrupt: ignored

In [50]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering

# Load NER pipeline for identifying hotel names
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", tokenizer="dslim/bert-base-NER")

# Load pre-trained language model for answering questions
model_name = "distilbert-base-uncased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

def get_hotel_info(context):
    # Identify hotel names in context using NER
    hotel_names = []
    ner_results = ner_pipeline(context)
    for result in ner_results:
        if result['entity'] == 'HOTEL':
            hotel_names.append(result['word'])
    
    # Loop through each identified hotel and answer some basic questions
    for hotel_name in hotel_names:
        print("Hotel: ", hotel_name)
        questions = [
            "What is the address of " + hotel_name + "?",
            "What are the amenities at " + hotel_name + "?",
            "What are the room rates at " + hotel_name + "?"
        ]
        for question in questions:
            inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
            input_ids = inputs["input_ids"].tolist()[0]

            # Get the model's answer to the question
            outputs = model(**inputs)
            answer_start = outputs.start_logits.argmax()
            answer_end = outputs.end_logits.argmax() + 1
            answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

            print(question)
            print(answer)
        print("=" * 50)

# Example context containing hotel information
context = "The Hilton Garden Inn is a popular hotel located in the heart of downtown. It offers a variety of amenities, including an indoor pool, fitness center, and on-site restaurant. The hotel is known for its spacious rooms and excellent service."

# Get information about hotels in the context
get_hotel_info(context)

In [65]:
from transformers import pipeline

context = "1 KING BED/SMOKING/Clock Radio/AM-FM//Air Conditioning/Tea and Coffee in Room/Coffee Maker/Interior Corridor/Hair Dryer"

qa = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad")

question = "What is room type?"

result = qa(question=question, context=context)

print(result["answer"])

Coffee Maker/Interior Corridor/Hair Dryer


In [66]:
question = "what is bed type?"

result = qa(question=question, context=context)

print(result["answer"])

KING


In [67]:
question = "does it have ocean view?"

result = qa(question=question, context=context)

print(result["answer"])

1 KING BED


In [68]:
question = "does hotel have free breakfast"

result = qa(question=question, context=context)

print(result["answer"])

Coffee


In [80]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, ne_chunk

# Define a function to preprocess the room descriptions
def preprocess(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Tag the part-of-speech of the tokens
    pos_tags = pos_tag(tokens)
    
    # Identify named entities in the text
    named_entities = ne_chunk(pos_tags)
    
    # Extract relevant features from the text
    features = []
    for chunk in named_entities:
        if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
            features.append('person')
        elif hasattr(chunk, 'label') and chunk.label() == 'ORGANIZATION':
            features.append('organization')
        elif hasattr(chunk, 'label') and chunk.label() == 'GPE':
            features.append('location')
        elif chunk[0].isdigit():
            features.append('number')
        else:
            features.append(chunk[0])
            
    return features

# Example room descriptions
room1 = 'Superior King Room with Balcony'
room2 = 'Superior Room, 1 King-Size Bed, Balcony'
room3 = 'Superior King Room with Balcony'
room4 = 'Luxury Room, King-Size Bed, Balcony View Of The Swimming Pool'

# Preprocess the room descriptions
features1 = preprocess(room1)
features2 = preprocess(room2)
features3 = preprocess(room3)
features4 = preprocess(room4)

# Print the extracted features for each room description
print('Room 1:', features1)
print('Room 2:', features2)
print('Room 3:', features3)
print('Room 4:', features4)

Room 1: ['superior', 'king', 'room', 'balcony']
Room 2: ['superior', 'room', ',', 'number', 'king-size', 'bed', ',', 'balcony']
Room 3: ['superior', 'king', 'room', 'balcony']
Room 4: ['luxury', 'room', ',', 'king-size', 'bed', ',', 'balcony', 'view', 'swimming', 'pool']


In [79]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/n

True

In [82]:
import spacy

# Load the language model
nlp = spacy.load("en_core_web_sm")

# Define the room descriptions
room1 = "Superior King Room with Balcony"
room2 = "Superior Room, 1 King-Size Bed, Balcony"
room3 = "Superior King Room with Balcony"
room4 = "Luxury Room, King-Size Bed, Balcony View Of The Swimming Pool"

# Define a function to extract relevant features from a room description
def extract_features(description):
    doc = nlp(description)
    features = []
    for token in doc:
        # Check if the token is a noun, adjective, or number
        if token.pos_ in ["NOUN", "ADJ", "NUM"]:
            features.append(token.text)
        # Check if the token is a named entity
        elif token.ent_type_:
            features.append(token.ent_type_)
    return features

# Extract features from each room description
features1 = extract_features(room1)
features2 = extract_features(room2)
features3 = extract_features(room3)
features4 = extract_features(room4)

# Print the extracted features
print("Features for room 1:", features1)
print("Features for room 2:", features2)
print("Features for room 3:", features3)
print("Features for room 4:", features4)

Features for room 1: ['Superior']
Features for room 2: ['1', 'ORG', 'ORG', 'ORG', 'ORG', 'GPE']
Features for room 3: ['Superior']
Features for room 4: ['PERSON', 'PERSON', 'ORG', 'ORG', 'ORG', 'ORG']


Can Post tagging help?

In [83]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Sample room descriptions
room1 = "Superior King Room with Balcony"
room2 = "Superior Room, 1 King-Size Bed, Balcony"
room3 = "Luxury Room, King-Size Bed, Balcony View Of The Swimming Pool"
room4 = "Superior King Room with Balcony"

# Tokenize and part-of-speech tag each room description
tokens1 = word_tokenize(room1)
pos_tags1 = pos_tag(tokens1)

tokens2 = word_tokenize(room2)
pos_tags2 = pos_tag(tokens2)

tokens3 = word_tokenize(room3)
pos_tags3 = pos_tag(tokens3)

tokens4 = word_tokenize(room4)
pos_tags4 = pos_tag(tokens4)

# Print out the tokens and part-of-speech tags for each room description
print("Tokens and POS tags for room 1:")
print(pos_tags1)
print()

print("Tokens and POS tags for room 2:")
print(pos_tags2)
print()

print("Tokens and POS tags for room 3:")
print(pos_tags3)
print()

print("Tokens and POS tags for room 4:")
print(pos_tags4)

Tokens and POS tags for room 1:
[('Superior', 'JJ'), ('King', 'NNP'), ('Room', 'NNP'), ('with', 'IN'), ('Balcony', 'NNP')]

Tokens and POS tags for room 2:
[('Superior', 'JJ'), ('Room', 'NNP'), (',', ','), ('1', 'CD'), ('King-Size', 'JJ'), ('Bed', 'NNP'), (',', ','), ('Balcony', 'NNP')]

Tokens and POS tags for room 3:
[('Luxury', 'NN'), ('Room', 'NNP'), (',', ','), ('King-Size', 'NNP'), ('Bed', 'NNP'), (',', ','), ('Balcony', 'NNP'), ('View', 'NNP'), ('Of', 'IN'), ('The', 'DT'), ('Swimming', 'NNP'), ('Pool', 'NNP')]

Tokens and POS tags for room 4:
[('Superior', 'JJ'), ('King', 'NNP'), ('Room', 'NNP'), ('with', 'IN'), ('Balcony', 'NNP')]


In [87]:
import spacy

# Load a pre-trained NLP model
nlp = spacy.load("en_core_web_sm")

# Define a rule-based function to match rooms with similar descriptions
def match_rooms(room1, room2):
    # Remove punctuation and whitespace from both room descriptions
    room1_clean = room1.lower().translate(str.maketrans("", "", ".,;:?!()[]{}'\""))
    room2_clean = room2.lower().translate(str.maketrans("", "", ".,;:?!()[]{}'\""))

    # Compare the cleaned room descriptions to see if they are similar
    if room1_clean == room2_clean:
        return True
    else:
        # Use the NLP model to extract relevant features from the room descriptions
        room1_doc = nlp(room1)
        room2_doc = nlp(room2)

        # Check if the room descriptions have the same number of tokens
        if len(room1_doc) == len(room2_doc):
            # Calculate the similarity score between the two room descriptions
            similarity_score = room1_doc.similarity(room2_doc)

            # If the similarity score is high enough, consider the rooms to be a match
            if similarity_score >= 0.8:
                return True

    # If none of the above conditions are met, the rooms do not match
    return False

# Example usage:
room1 = "Superior King Room with Balcony"
room2 = "Superior Room, 1 King-Size Bed, Balcony"
room3 = "Superior King Room with Balcony"
room4 = "Luxury Room, King-Size Bed, Balcony View Of The Swimming Pool"

# Use the rule-based function to check which rooms match
if match_rooms(room1, room2):
    print("Room 1 and Room 2 represent the same description.")
else:
    print("Room 1 and Room 2 do not represent the same description.")

if match_rooms(room1, room3):
    print("Room 1 and Room 3 represent the same description.")
else:
    print("Room 1 and Room 3 do not represent the same description.")

if match_rooms(room1, room4):
    print("Room 1 and Room 4 represent the same description.")
else:
    print("Room 1 and Room 4 do not represent the same description.")

if match_rooms(room3, room4):
    print("Room 3 and Room 4 represent the same description.")
else:
    print("Room 3 and Room 4 do not represent the same description.")

if match_rooms(room2, room3):
    print("Room 2 and Room 3 represent the same description.")
else:
    print("Room 2 and Room 3 do not represent the same description.")

Room 1 and Room 2 do not represent the same description.
Room 1 and Room 3 represent the same description.
Room 1 and Room 4 do not represent the same description.
Room 3 and Room 4 do not represent the same description.
Room 2 and Room 3 do not represent the same description.
